In [16]:
import osmnx as ox
import networkx as nx
import gpxpy
import gpxpy.gpx
import itertools

In [2]:
def fetch_road_network(north, south, east, west):
    """Fetch the road network within the given boundary."""
    G = ox.graph_from_bbox(north, south, east, west, network_type='bike')
    return G

In [13]:
def eulerize_graph(G):
    """Make a graph Eulerian by adding edges."""
    # Find nodes with odd degree
    odd_degree_nodes = [node for node, degree in G.degree() if degree % 2 == 1]
    
    # Compute shortest paths between odd nodes
    odd_node_pairs = list(itertools.combinations(odd_degree_nodes, 2))
    shortest_paths = {}
    for u, v in odd_node_pairs:
        shortest_paths[(u, v)] = nx.shortest_path_length(G, u, v, weight='length')

    # Determine pairs to connect
    g = nx.Graph()
    g.add_weighted_edges_from([(u, v, shortest_paths[(u, v)]) for u, v in odd_node_pairs])
    mwcm = nx.max_weight_matching(g, maxcardinality=True)

    # Augment the graph
    for u, v in mwcm:
        path = nx.shortest_path(G, u, v, weight='length')
        path_edges = list(zip(path[:-1], path[1:]))
        G.add_edges_from(path_edges)

    return G

In [14]:
def create_gpx_from_route(route, G):
    """Create a GPX file from the given route."""
    gpx = gpxpy.gpx.GPX()

    # Create first track in our GPX
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)

    # Create first segment in our GPX track
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)

    for node in route:
        lon, lat = G.nodes[node]['x'], G.nodes[node]['y']
        gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(lat, lon))

    return gpx.to_xml()

In [19]:
north, south, east, west = 42.399037, 42.37, -71.055719, -71.08
# north, south, east, west = 42.399037, 42.331741, -71.055719, -71.160084  # Example boundary (around Boston)

G = fetch_road_network(north, south, east, west)
G_eulerized = eulerize_graph(G)
eulerian_circuit = list(nx.eulerian_circuit(G_eulerized))
route = [u for u, v in eulerian_circuit]

gpx_data = create_gpx_from_route(route, G_eulerized)

with open("route.gpx", "w") as f:
    f.write(gpx_data)

NetworkXNoPath: Node 71918742 not reachable from 61344475